In [20]:
import requests
from IPython.core.display import HTML

In [22]:
%%writefile server.py

from flask import Flask, request, render_template, redirect, session
import cv2
import numpy as np
import datetime
import requests
from IPython.core.display import HTML

app = Flask(__name__)

@app.route('/')
def index():
    html = """
    <h1>안녕하세요</h1>
    """

    return html

datas = [45.7, 45, 10, 70.8]

@app.route('/signal') # 데이터 송수신
def signal():
    global datas
    datas.append( float(request.args.get("data")) )
    return str(datas)

@app.route('/view') # 데이터를 모니터링 하는 앱
def view():
    global datas
    
#     labels = ["" for d in datas]
    labels = [i+1 for i in range(len(datas))]
    
    ctx = {'title':'그래프', 
           'labels':labels,
           'data':datas,
          }
    
    return render_template('view.html', ctx=ctx)

@app.route('/upload')
def upload():
    return ""


if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, port=8000)

Overwriting server.py


In [1]:

%%writefile server.py

from flask import Flask, request, render_template, redirect, session, make_response
import cv2
import numpy as np
import datetime
import requests
import qrcode
from pyzbar.pyzbar import decode

app = Flask(__name__)

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('haarcascade_eye.xml')

@app.route('/')
def index():
    html = """
        보행자 검출 Test
    """
    return html


@app.route('/upload', methods=["post"])
def upload():
    f = request.files['file1']
    filename = "./static/" + f.filename
    f.save(filename)
    
    img = cv2.imread(filename)
#     img = cv2.resize(img, dsize=(0,0), fx=0.25, fy=0.25)
    
    faces = face_classifier.detectMultiScale(img)
    
    if len(faces) > 0:
        for (x1, y1, w1, h1) in faces:
            cv2.rectangle(img, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 255), 2)

        retval, buffer = cv2.imencode('.jpg', img)

        response = make_response(buffer.tobytes())
        response.headers['Content-Type'] = 'image/jpg'
    
        return response
    
    code = decode(img)
    url = code[0].data.decode('utf-8')
    
    
    return f"<a href='{url}'>QR코드가기</a>"

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, port=8000)

Overwriting server.py


In [1]:
%%writefile server.py
import cv2
import numpy as np
import io
import random
from flask import Flask, request, render_template, make_response, Response

app = Flask(__name__)

vc = cv2.VideoCapture('vtest.avi')

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('haarcascade_eye.xml')


datas = []

@app.route('/')
def index():
    html = """
    <h1>사람 찾기</h1>
    <button type=button id=haar>Haar</button>
    <button type=button id=hog>HOG</button>
    <img src=/video_feed width=320 height=240>
    """
    return  html


def gen():
    while True:
        read_return_code, frame = vc.read()
        
        detected, _ = hog.detectMultiScale(frame)
        
        for (x, y, w, h) in detected: 
            c = (255,100,100)
            cv2.rectangle(frame, (x, y), (x + w, y + h), c, 3)
        
        rect = detected
        datas.append(len(rect))
        
        encode_return_code, image_buffer = cv2.imencode('.jpg', frame)
        io_buf = io.BytesIO(image_buffer)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + io_buf.read() + b'\r\n')
        

@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    
    return Response(
        gen(),
        mimetype='multipart/x-mixed-replace; boundary=frame'
    )

@app.route('/view') # 데이터를 모니터링 하는 앱
def view():
    global datas
    
#     labels = ["" for d in datas]
    labels = [i+1 for i in range(len(datas))]
    
    ctx = {'title':'그래프', 
           'labels':labels,
           'data':datas,
          }
    
    return render_template('view.html', ctx=ctx)

if __name__ == '__main__':
     app.run(host='0.0.0.0', debug=True, port=8000)      


Writing server.py


### generator

In [8]:
def fun():
    yield 10
    yield 20
    yield 30

for i in fun():
    print(i)

10
20
30


In [9]:
a = fun()
print(next(a))
print(next(a))
print(next(a))

10
20
30


In [12]:
def fun():
    while True:
        yield 10
        yield 20
        yield 30

a = fun()
print(next(a))
print(next(a))
print(next(a))
print(next(a))
print(next(a))
print(next(a))
print(next(a))
print(next(a))
print(next(a))

10
20
30
10
20
30
10
20
30


In [13]:
print(next(a))

10


In [2]:
%%writefile server.py
import cv2
import numpy as np
import io
import random

from flask import Flask, request, render_template, make_response, Response
app = Flask(__name__)

vc = cv2.VideoCapture('vtest.avi')

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

datas = []

def detectHuman(frame) :
    detected, _ = hog.detectMultiScale(frame)

    for (x, y, w, h) in detected:
        c = (random.randint(0, 255),
             random.randint(0, 255), 
             random.randint(0, 255))
        cv2.rectangle(frame, (x, y), (x + w, y + h), c, 3)
    return detected    

@app.route('/')
def index():
    return  render_template("human.html")

def gen():
    global vc
    global datas
    
    while True:
        read_return_code, frame = vc.read()
        
        if not read_return_code :
            vc = cv2.VideoCapture('vtest.avi')
            datas = []
            break;
        
        rect = detectHuman(frame)
        
        datas.append(len(rect))
        
        
        encode_return_code, image_buffer = cv2.imencode('.jpg', frame)
        io_buf = io.BytesIO(image_buffer)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + io_buf.read() + b'\r\n')


@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(
        gen(),
        mimetype='multipart/x-mixed-replace; boundary=frame'
    )

@app.route('/view')
def view():
    global datas
    return str(datas)
    
if __name__ == '__main__':
     app.run(host='0.0.0.0', debug=True, port=8000)      

Overwriting server.py
